In [36]:
import pandas as pd

In [70]:
df19 = pd.read_csv('notebooks/datos_desag_2019_cuentas.csv')
df24 = pd.read_csv('notebooks/datos_desag_2024_cuentas.csv')
ipc = pd.read_csv('notebooks/ipc_basic.csv')

In [71]:
ipc = ipc[['Año', 'Indice']]

In [72]:
filtro = ipc[ipc['Año'].isin([2019, 2024])]
#filtro['Año'] += 1

In [73]:
filtro['Indice_24'] = filtro['Indice'] * 100 / (137.72)

C:\Users\user\AppData\Local\Temp\ipykernel_9744\3823183924.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro['Indice_24'] = filtro['Indice'] * 100 / (137.72)


In [74]:
filtro

,Año,Indice,Indice_24
6,2019,100.00,72.611095
11,2024,137.72,100.000000


In [75]:
df24.columns

Index(['Cuenta', 'Subcuenta', 'Objeto', 'Ordinal', 'Aporte nacional',
       'Recursos propios', 'TOTAL', 'Código de entidad', 'Tipo de gasto',
       'Entidad', 'Código de sector', 'Sector', 'Unidad', 'Código de unidad',
       'Cuenta_n', 'Subcuenta_n', 'Objeto_n', 'Ordinal_n'],
      dtype='object')

In [76]:
df19.columns = df24.columns

In [77]:
df19['Año'] = 2019
df24['Año'] = 2024

In [78]:
df = pd.concat([df19, df24], ignore_index=True)

In [79]:
df = df.merge(filtro[['Año', 'Indice_24']])

In [80]:
df['Total_24'] = df['TOTAL'] / (df['Indice_24'] / 100)

In [81]:
df.pivot_table(index=['Tipo de gasto'],
            columns='Año', values='Total_24', aggfunc='sum').assign(diff=lambda x: x[2024] - x[2019])

Año,2019,2024,diff
Tipo de gasto,,,
Deuda,7.152463e+13,9.452185e+13,2.299722e+13
Funcionamiento,2.206764e+14,3.082237e+14,8.754727e+13
Inversión,6.449006e+13,9.985131e+13,3.536125e+13


In [82]:
sgp = df[(df['Cuenta'] == 'Transferencias corrientes') &
   (df['Subcuenta'] == 'A entidades del gobierno') & 
   (df['Objeto'] == 'Sistema general de participaciones')]

In [118]:
df[df['Año'] == 2024]['Código de entidad'].nunique()

163

In [121]:
(sgp.pivot_table(index='Ordinal',
                columns='Año',
                values='Total_24',
                aggfunc='sum')
                .assign(cagr=lambda x: (x[2024]/x[2019]) ** (1/5) - 1) * 100)

Año,2019,2024,cagr
Ordinal,,,
Agua potable y saneamiento básico,2.932633e+14,3.624436e+14,4.326955
Fonpet - asignaciones especiales,NaN,2.281446e+14,NaN
Municipios de la ribera del río magdalena - asignaciones especiales,4.525669e+12,5.593265e+12,4.326955
Participación para educación - distribución previo concepto dnp,3.269535e+15,3.968590e+15,3.951326
Participación para propósito general,6.299731e+14,7.785825e+14,4.326955
Participación para salud - distribución previo concepto dnp,1.426951e+15,1.639420e+15,2.814948
Programas de alimentación escolar - asignaciones especiales,2.828543e+13,3.495791e+13,4.326955
Resguardos indígenas - asignaciones especiales,2.941685e+13,3.635622e+13,4.326955


In [122]:
inc = pd.read_csv('ingresos_pgn.csv')

In [129]:
inc[inc['Año'].isin([2019, 2024]) & (inc['Ingreso_alt'] == 'INGRESOS CORRIENTES DE LA NACIÓN')]['Valor_24']

1209    1.993836e+14
1896    3.174002e+14
Name: Valor_24, dtype: float64

In [131]:
((3.174002e+14 / 1.993836e+14) ** (1/5) - 1 ) * 100

9.744699224323682

In [86]:
sgp.loc[sgp['Ordinal'] == 'Municipios de la ribera del rio magdalena - asignaciones especiales', 'Ordinal'] = 'Municipios de la ribera del río magdalena - asignaciones especiales'

In [104]:
func = df[(df['Tipo de gasto'] == 'Funcionamiento') & (df['Año'] == 2019)]

In [115]:
df[df['Ordinal'] == 'Fondo nacional de pensiones de las entidades territoriales ley 549 de 1999 (de pensiones)']

,Cuenta,Subcuenta,Objeto,Ordinal,Aporte nacional,Recursos propios,TOTAL,Código de entidad,Tipo de gasto,Entidad,...,Sector,Unidad,Código de unidad,Cuenta_n,Subcuenta_n,Objeto_n,Ordinal_n,Año,Indice_24,Total_24
475,Transferencias corrientes,Prestaciones sociales,Prestaciones sociales asumidas por el gobierno,Fondo nacional de pensiones de las entidades t...,1.576380e+11,NaN,1.576380e+11,1301,Funcionamiento,Ministerio de hacienda y crédito público,...,Hacienda,NaN,NaN,3.0,4.0,3.0,1.0,2019,72.611095,2.170991e+11
3821,Transferencias corrientes,Prestaciones para cubrir riesgos sociales,Prestaciones sociales asumidas por el gobierno,Fondo nacional de pensiones de las entidades t...,1.234510e+11,NaN,1.234510e+11,1301,Funcionamiento,Ministerio de hacienda y crédito público,...,Hacienda,NaN,NaN,3.0,4.0,3.0,001,2024,100.000000,1.234510e+11


In [ ]:
df['Sector']

In [135]:
ambiente = df[df['Entidad'] == 'Ministerio de ambiente y desarrollo sostenible']

In [150]:
ambiente[ambiente['Cuenta'] == 'Gastos de personal'].pivot_table(index='Subcuenta',
                     aggfunc='sum',
                     values='Total_24',
                     columns='Año').assign(diff=lambda x: ((x[2024]/x[2019])**(1/5) - 1) * 100)

Año,2019,2024,diff
Subcuenta,,,
Planta de personal permanente,1.036054e+11,1.767140e+11,11.269904
